In [1]:
include("gradient-boosting.jl")

rmse (generic function with 1 method)

In [2]:
using CSV,DataFrames,Printf,BenchmarkTools,LinearAlgebra, Statistics, Compat

In [3]:
print("Load data...")
df_train = CSV.read("./data/regression.train"; delim="\t",header=false)
df_test = CSV.read("./data/regression.test"; delim="\t",header=false)



Load data...

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `using DataFrames; CSV.read(input, DataFrame; kw...)
│   caller = ip:0x0
└ @ Core :-1


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.644,0.247,-0.447,0.862,0.374,0.854,-1.126,-0.79
2,0,0.385,1.8,1.037,1.044,0.349,1.502,-0.966,1.734
3,0,1.214,-0.166,0.004,0.505,1.434,0.628,-1.174,-1.23
4,1,0.42,1.111,0.137,1.516,-1.657,0.854,0.623,1.605
5,0,0.897,-1.703,-1.306,1.022,-0.729,0.836,0.859,-0.333
6,0,0.756,1.126,-0.945,2.355,-0.555,0.889,0.8,1.44
7,0,1.141,-0.741,0.953,1.478,-0.524,1.197,-0.871,1.689
8,1,0.606,-0.936,-0.384,1.257,-1.162,2.719,-0.6,0.1
9,1,0.603,0.429,-0.279,1.448,1.301,1.008,2.423,-1.295


In [4]:
y_train = convert(Array,df_train[1])
y_test = convert(Array,df_test[1])
X_train = convert(Matrix,deletecols!(df_train, 1))
X_test =  convert(Matrix,deletecols!(df_test, 1))

500×28 Array{Float64,2}:
 0.644   0.247  -0.447  0.862   0.374  …  0.98   0.875  0.581  0.905  0.796
 0.385   1.8     1.037  1.044   0.349     0.981  0.803  0.813  1.149  1.116
 1.214  -0.166   0.004  0.505   1.434     1.043  0.931  1.058  0.744  0.696
 0.42    1.111   0.137  1.516  -1.657     1.105  0.73   0.148  1.231  1.234
 0.897  -1.703  -1.306  1.022  -0.729     0.984  1.43   2.198  1.934  1.684
 0.756   1.126  -0.945  2.355  -0.555  …  0.988  0.803  0.234  0.822  0.911
 1.141  -0.741   0.953  1.478  -0.524     1.747  1.358  2.479  1.491  1.223
 0.606  -0.936  -0.384  1.257  -1.162     0.985  1.574  4.323  2.263  1.742
 0.603   0.429  -0.279  1.448   1.301     1.281  0.763  0.652  0.827  0.785
 1.171  -0.962   0.521  0.841  -0.315     0.987  0.846  1.293  0.899  0.756
 1.392  -0.358   0.235  1.494  -0.461  …  1.17   1.362  0.959  2.056  1.842
 1.024   1.076  -0.886  0.851   1.53      1.063  1.193  0.777  0.964  0.807
 0.89   -0.76    1.182  1.369   0.751     0.998  1.126  0.788  

In [5]:
train_data = Dataset(X_train, y_train)
eval_data = Dataset(X_test, y_test)

Dataset([0.644 0.247 … 0.905 0.796; 0.385 1.8 … 1.149 1.116; … ; 3.512 -1.094 … 1.626 1.349; 0.904 1.248 … 1.012 0.961], [1, 0, 0, 1, 0, 0, 0, 1, 1, 0  …  1, 0, 1, 0, 1, 1, 0, 1, 0, 0])

In [6]:
params=Dict("gamma"=> 0.,
                       "lambda"=> 1.,
                       "min_split_gain"=> 0.1,
                       "max_depth"=> 5,
                       "learning_rate"=> 0.3,
                       )

Dict{String,Real} with 5 entries:
  "max_depth"      => 5
  "gamma"          => 0.0
  "lambda"         => 1.0
  "learning_rate"  => 0.3
  "min_split_gain" => 0.1

In [12]:
print("Start training...")
gbt = GradientBoostedTree()
train!(gbt,params,train_data,25,eval_data,5,l2_loss)

Start training...Training until validation scores don't improve for 5 rounds. 
Iter 1, Train's Loss: 1315.928465, Valid's Loss: 99.477324, Elapsed: 0.904038 secs 
Iter 2, Train's Loss: 3343.181945, Valid's Loss: 240.157691, Elapsed: 1.169416 secs 
Iter 3, Train's Loss: 3260.506033, Valid's Loss: 234.623938, Elapsed: 0.903499 secs 
Iter 4, Train's Loss: 3242.527208, Valid's Loss: 233.600982, Elapsed: 1.163453 secs 
Iter 5, Train's Loss: 3245.592307, Valid's Loss: 233.832391, Elapsed: 0.894928 secs 
Iter 6, Train's Loss: 3236.210456, Valid's Loss: 233.187566, Elapsed: 1.156144 secs 
Early stopping, best iteration is: 
Iter 1.000000, Train's Loss: 99.477324Training finished. Elapsed: 6.193106 secs

In [13]:
print("Start predicting...")
y_pred = Float64[]
for i  in len(X_test)
    push!(y_pred,predict(gbt,X_test[i,:]))
end

@printf("The rmse of prediction is: %f", rmse(y_test,y_pred))

Start predicting...The rmse of prediction is: 0.555729